In [ ]:
# LangChain expression language is a way of grouping chains out of individual components called runnables
# Runnable support invoke(), stream(), batch() and their asynch analogs
# batch() can handle more than one input

In [1]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser

In [5]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [7]:
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [15]:
chat_template = ChatPromptTemplate.from_messages([('human', "I've recently adapted a {pet}. Could you suggest a {pet} name? \n" + list_instructions)])

In [17]:
chat_template

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adapted a {pet}. Could you suggest a {pet} name? \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`"), additional_kwargs={})])

In [5]:
chat = ChatOpenAI ( model='gpt-4', 
                   seed=365 , 
                   temperature=0, 
                   max_completion_tokens=100)

In [21]:
list_output_parser = CommaSeparatedListOutputParser()

In [ ]:
# Now we have all the components to build an expression language chain - a chat template, a chat model and a chat parser

In [27]:
chat_template_result = chat_template.invoke({'pet':'dog'})

In [29]:
chat_result = chat.invoke(chat_template_result)

In [31]:
list_output_parser.invoke(chat_result)

['Max',
 'Bella',
 'Charlie',
 'Daisy',
 'Buddy',
 'Luna',
 'Rocky',
 'Molly',
 'Jack',
 'Sadie']

In [33]:
chain = chat_template | chat | list_output_parser

In [35]:
chain.invoke({'pet':'dog'})

['Max',
 'Bella',
 'Charlie',
 'Daisy',
 'Buddy',
 'Luna',
 'Rocky',
 'Molly',
 'Jack',
 'Sadie']

In [37]:
chat_template_batch = ChatPromptTemplate.from_messages(['human', "I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?"])

In [39]:
batch_chain = chat_template_batch | chat

In [43]:
batch_chain.invoke({'pet':'dog', 'breed':'shepherd'})

AIMessage(content='Absolutely, training a shepherd dog can be a rewarding experience as they are known for their intelligence and eagerness to learn. Here are some tips:\n\n1. Start Early: Begin training your shepherd as soon as you bring them home. They are quick learners and starting early will help them understand basic commands and house rules.\n\n2. Consistency is Key: Be consistent with your commands and the behavior you expect. If you allow your dog to break rules occasionally, they will get confused about what they are allowed', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 30, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None}, id='run-b

In [ ]:
# invoke method doesnt allow for feeding several inputs at once if you want training tips for several pets. We can do in a loop but batching is better
#Batching invokes the cells in parallel

In [47]:
batch_chain.batch([{'pet':'dog', 'breed':'shepherd'},
                  {'pet':'dragon','breed':'night fury'}])

[AIMessage(content="Absolutely, training a shepherd dog can be a rewarding experience as they are known for their intelligence and eagerness to learn. Here are some tips:\n\n1. Start Early: Begin training your shepherd as soon as you bring them home. They are quick learners and starting early will help them understand basic commands and house rules.\n\n2. Consistency is Key: Be consistent with your commands and the behavior you expect. If you allow your dog to do something once, they will think it's always acceptable.\n\n3", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 30, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None}, id='run-7b9e20f1-bc29

In [ ]:
# if you invoke each pet seprately and then use batch, you will see time efficiency using the following commands

In [49]:
%%time
batch_chain.invoke({'pet':'dog', 'breed':'shepherd'})

CPU times: user 14.3 ms, sys: 3.33 ms, total: 17.7 ms
Wall time: 5.02 s


AIMessage(content='Absolutely, training a shepherd dog can be a rewarding experience as they are known for their intelligence and eagerness to learn. Here are some tips:\n\n1. Start Early: Begin training your shepherd as soon as you bring them home. They are quick learners and starting early will help them understand basic commands and house rules.\n\n2. Consistency is Key: Be consistent with your commands and the behavior you expect. If you allow your dog to break rules occasionally, they will get confused about what they are allowed', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 30, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None}, id='run-0

In [51]:
%%time
batch_chain.batch([{'pet':'dog', 'breed':'shepherd'},
                  {'pet':'dragon','breed':'night fury'}])

CPU times: user 27.6 ms, sys: 5.84 ms, total: 33.4 ms
Wall time: 5.61 s


[AIMessage(content='Absolutely, training a shepherd dog can be a rewarding experience as they are known for their intelligence and eagerness to learn. Here are some tips:\n\n1. Start Early: Begin training your shepherd as soon as you bring them home. They are quick learners and starting early will help them understand basic commands and house rules.\n\n2. Consistency is Key: Be consistent with your commands and the behavior you expect. If you allow your dog to break rules occasionally, they will get confused about what they are allowed', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 30, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None}, id='run-

In [ ]:
# Passing several inputs at once to a batch() method is much more efficient that invoking a chain many times in sequence.
# this can scale up significantly with more dictionaries and lengthier responses

In [53]:
batch_chain.stream({'pet':'dog', 'breed':'shepherd'})

<generator object RunnableSequence.stream at 0x114634130>

In [ ]:
# Generator functions are like loop function. They preserve and expose the local state. uses a yield statements that produces a seq of values
# instead of a return statement that exits the function
# they are single use iterator, cannot be resused. Ideal for streaming responses from llm with limited mem

In [ ]:
response = batch_chain.invoke({'pet':'dog', 'breed':'shepherd'})
next(response) # This method shows you AIMEssageChunk, one by one in the iterator. You can run it as many times as the chat max tokens

In [ ]:
for i in response:
    print(i.content, end='')

# This code streams the text in the output

In [ ]:
# What are the different types of lang chain objects that can be integrated into a chain and what is the resulting chain

In [55]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [ ]:
# This class implements a sequence of runnables where the output of each is input to the next. Chat model, template and and output is instance of runnable sequence

In [ ]:
# therefore inherits the invoke, batch, stream method, can be chained. Chains are also runnables and can also be chained
# Retriever and Tools are also runnables

In [7]:
from langchain_core.runnables import RunnablePassthrough

In [8]:
runnable = RunnablePassthrough()
runnable.invoke(input='Hi')

'Hi'

In [9]:
# This is langchain's identity function. How do we now pipe chains
# Lets create a chain that lists the most essential tools for a given profession
# Feed the output to a second chain that suggest effective strategies for mastering these tools

In [10]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

In [11]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [12]:
chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for efficiently learning and mastering them:
{tools}
''')

In [13]:
chat_template_strategy

ChatPromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, template='\nConsidering the tools provided, develop a strategy for efficiently learning and mastering them:\n{tools}\n'), additional_kwargs={})])

In [14]:
string_parse = StrOutputParser()

In [15]:
chain_tools = chat_template_tools | chat | string_parse

In [16]:
chain_strategy = chat_template_strategy | chat | string_parse

In [17]:
print(chain_tools.invoke({'job title':'data scientist'}))

1. Python
2. R Programming
3. SQL
4. Tableau
5. Hadoop


In [18]:
print(chain_strategy.invoke({'tools':
 '''1. Python
2. R Programming
3. SQL
4. Tableau
5. Hadoop   
 '''   
}))

1. Python: Start with the basics of Python, such as variables, data types, operators, and control flow. Once you have a good understanding of these, move on to more complex topics like functions, classes, and error handling. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding regularly on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming: Begin with the basics of


In [19]:
# Now how do we merge the two chains into one? Use the RunnablePassthrough class
# A dictionary specifying the job title is fed to chat_template_tools -> chat_prompt_value object --> chat --> AIMessage --> string_parser
# --> RunnablePassthrough --> tools mapped to the string
# Invking chain_strategy with the output of the chain_tools will connect the two runnables.


In [20]:
chain_combined = chain_tools | chain_strategy

In [21]:
print( chain_combined.invoke ({'job title':'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects or contribute to open-source projects to gain practical experience. 

2. R Programming: Begin with the basics of R, such as data types, vectors, matrices, lists, and data frames. Use online resources


In [22]:
chain_long = chat_template_tools| chat | string_parse | {'tools': RunnablePassthrough()} | chat_template_strategy | chat  | string_parse

In [23]:
# python grandalf library helps us understand the process inside the chain

In [24]:
#pip install grandalf

Note: you may need to restart the kernel to use updated packages.


In [46]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

In [ ]:
# start with 2 chains. First suggest 2 intermediate programming level books and the second suggesting three projects on the same language & level

In [ ]:
# Both chains accept one input which is the language
# We want to run both chains parallely and study the graph and execution time

In [48]:
chat_template_books = ChatPromptTemplate.from_template('''
Suggest three of the best Intermediate level {programming language} books.
Answer only by listing the books.
''')

chat_template_projects = ChatPromptTemplate.from_template('''
Suggest three interesting {programming language} projects suitable for Intermediate level programmers.
Answer only by listing the projects.
''')

In [50]:
from langchain_core.runnables import RunnableParallel

In [52]:
str_par = StrOutputParser()

In [54]:
chain_books = chat_template_books | chat | str_par
chain_projects = chat_template_projects | chat | str_par

In [56]:
chain_prarallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [58]:
chain_prarallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Weather Forecasting Application using APIs\n2. Developing a Web Scraper for E-commerce Websites\n3. Creating a Personal Finance Tracker/Budgeting Tool'}

In [62]:
chain_prarallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [ ]:
# batch() allowed us to invoke the same Runnable with different input values
# RunnableParallel allowed us to invoke different runnables with same input values

In [ ]:
%%time
chain_books.invoke({'programming language':'Python'})

In [ ]:
%%time
chain_projects.invoke({'programming language':'Python'})

In [ ]:
%%time
chain_prarallel.invoke({'programming language':'Python'})

In [72]:
# Now take the output which is books and projects from the RunnableParallel and pass them to a new runnable to estimate the time to 
#complete the books and projects


In [78]:
chat_template_time = ChatPromptTemplate.from_template(
    '''
    I am an intermediate level programmer.
    Consider the following literature:
    {books}
    Consider the folllowing projects:
    {projects}
    Roughly how much time would it take me to complete the books and projects?
    '''
)

In [ ]:
# output from chain_prarallel.invoke({'programming language':'Python'}) should be fed into a new Runnable project

In [80]:
chain_time = RunnableParallel({'books':chain_books, 'projects':chain_projects}) | chat_template_time | chat | str_par

In [84]:
print(chain_time.invoke({'programming language':'Python'}))

The time it takes to complete the books and projects can vary greatly depending on several factors such as your reading speed, comprehension level, the complexity of the projects, and the amount of time you can dedicate to them each day. 

However, as a rough estimate:

1. "Fluent Python: Clear, Concise, and Effective Programming" - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around 2-3


In [86]:
chain_time.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [ ]:
# chain_time = RunnableParallel({'books':chain_books, 'projects':chain_projects}) | chat_template_time | chat | str_par
# SAME AS chain_time = {'books':chain_books, 'projects':chain_projects} | chat_template_time | chat | str_par
# RunnableParallel for the first input is wrapped automatically